In [1]:
import numpy as np
import pyvista as pv

# import panel as pn

# # Activate panel extension for pyvista support
# pn.extension('vtk')
pv.set_jupyter_backend('client')



In [2]:
# Read the data from the file
data = []
atoms = []
with open('test_evaluator.dat', 'r') as file:
    for line in file:
        values = line.strip().split()
        if len(values) == 3:
            atoms.append([float(values[0]), float(values[1]), float(values[2])])

        if len(values) == 4:
            data.append([float(values[0]), float(values[1]), float(values[2]), float(values[3])])

In [3]:
points = np.array(data)[:, :3]
viability = np.array(data)[:, 3]
print(points)
print(viability)

[[0.         0.         0.16666667]
 [0.         0.         0.19444445]
 [0.         0.         0.22222222]
 ...
 [0.94444442 0.94444442 0.77777779]
 [0.94444442 0.94444442 0.80555558]
 [0.94444442 0.94444442 0.83333331]]
[0.         0.         0.         ... 0.00812109 0.00804858 0.00718295]


In [4]:

# Create a regular grid (adjust the ranges and resolution as needed)
grid_x, grid_y, grid_z = np.mgrid[0:1:0.2/3.560745109, 0:1:0.2/3.560745109, 0:1:0.2/7.121490218]  # Regular 3D grid
# grid_x, grid_y, grid_z = np.mgrid[0:18:1, 0:18:1, 0:36:1]  # Regular 3D grid
print(grid_x)

[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.05616802 0.05616802 0.05616802 ... 0.05616802 0.05616802 0.05616802]
  [0.05616802 0.05616802 0.05616802 ... 0.05616802 0.05616802 0.05616802]
  [0.05616802 0.05616802 0.05616802 ... 0.05616802 0.05616802 0.05616802]
  ...
  [0.05616802 0.05616802 0.05616802 ... 0.05616802 0.05616802 0.05616802]
  [0.05616802 0.05616802 0.05616802 ... 0.05616802 0.05616802 0.05616802]
  [0.05616802 0.05616802 0.05616802 ... 0.05616802 0.05616802 0.05616802]]

 [[0.11233604 0.11233604 0.11233604 ... 0.11233604 0.11233604 0.11233604]
  [0.11233604 0.112336

In [5]:
# Convert known points to grid indices (round to nearest grid point)
grid_indices = np.rint(points / [0.2/3.560745109, 0.2/3.560745109, 0.2/7.121490218]).astype(int)  # Scale points to match grid resolution
print(grid_indices)

[[ 0  0  6]
 [ 0  0  7]
 [ 0  0  8]
 ...
 [17 17 28]
 [17 17 29]
 [17 17 30]]


In [6]:
# Create an empty viability grid initialized to 0 (same shape as grid_x, grid_y, grid_z)
grid_viability = np.zeros_like(grid_x)

# Fill in the viability values at the known grid points
for i, idx in enumerate(grid_indices):
    # print(idx)
    # print(grid_viability[idx[0], idx[1], idx[2]], viability[i])
    grid_viability[idx[0], idx[1], idx[2]] = viability[i] * 100
    # print(grid_viability[idx[0], idx[1], idx[2]])

# Now you have a grid with 0s where there are no data points and values where you have data

# Create a pyvista StructuredGrid from the regular grid
structured_grid = pv.StructuredGrid(grid_x, grid_y, grid_z)

# Add the viability data to the grid
structured_grid['viability'] = grid_viability.flatten(order="F")  # Flatten the array!


In [7]:
structured_grid.plot(show_edges=False, scalars='viability', cmap='coolwarm', show_scalar_bar=True)

Widget(value='<iframe src="http://localhost:53985/index.html?ui=P_0x308f38c10_0&reconnect=auto" class="pyvista…

In [8]:
# Generate the isosurface (e.g., where viability = 10.0)
contours = structured_grid.contour(isosurfaces=[0.01])

# Plot the isosurface
plotter = pv.Plotter()
plotter.add_mesh(contours, color='white')
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:53985/index.html?ui=P_0x3091b0610_1&reconnect=auto" class="pyvista…

In [9]:

# # Plotter object for interactive rendering
# plotter = pv.Plotter(notebook=True)

# # Callback for the slider
# def update_isosurface(value):
#     plotter.clear()
#     contours = structured_grid.contour(isosurfaces=[value])
#     plotter.add_mesh(contours, color='white')
#     plotter.render()

# # Add initial isosurface
# initial_value = 0.01
# contours = structured_grid.contour(isosurfaces=[initial_value])
# plotter.add_mesh(contours, color='red')

# # Panel slider widget for interactivity
# slider = pn.widgets.FloatSlider(name='Isosurface Level', start=0, end=np.max(viability), value=initial_value)

# # Link the slider with the callback function
# slider.param.watch(lambda event: update_isosurface(event.new), 'value')

# # Show plot and slider in a panel layout
# layout = pn.Column(slider, plotter.show(jupyter_backend='panel'))

# layout.show()

In [78]:
pl = pv.Plotter(off_screen=True)
# Create the point cloud as a `pv.PolyData` object
point_cloud = pv.PolyData(points)

# Add scalars for coloring the points based on their viability
point_cloud["viability"] = viability ** (1/3) * 10

# Use glyphs to represent each point as a small sphere
glyphs = point_cloud.glyph(scale=True, geom=pv.Sphere(radius=0.01))  # Adjust radius as needed

# Add the glyphs to the plotter
pl.set_scale(zscale=2)
pl.add_mesh(
    # glyphs,
    points,
    scalars = viability ** (1/3) * 10,
    render_points_as_spheres=True,
    # scalars="viability",
    show_scalar_bar=True,
    color=True,
    # opacity = [0.1, 0.2, 0.9],
    point_size=30.0,
    # opacity=0.8,  # You can try different opacity values
);
# contour = structured_grid.contour(isosurfaces=[0.01])
# pl.add_mesh(
#     points,
#     scalars=viability*100,
#     render_points_as_spheres=True,
#     style='points_gaussian',
#     show_scalar_bar=True,
#     # emissive=True,
#     # opacity=0.25,
#     point_size=1.0,
#     ambient=0.7,
#     color=True,
#     opacity = 1.0,#[0.05, 0.2, 0.9],
#     # use_transparency=True,
#     emissive=False,
# );

In [79]:

# pl.background_color = 'k'
pl.show(auto_close=False)

Widget(value='<iframe src="http://localhost:53985/index.html?ui=P_0x337c48b50_35&reconnect=auto" class="pyvist…

: 